# Pyomo Parameters

In [1]:
# Esta celda da el estilo al notebook
from IPython.core.display import HTML
css_style = r'..\style_1.css'
css_file = css_style
HTML(open(css_file, "r").read())

In [2]:
from pyomo.environ import *

## Declaring parameters

In order to declare a parameter, the model must be created:

In [3]:
model = ConcreteModel()

The simplest way is:

In [4]:
model.Z = Param()

In [5]:
model.Z.pprint()

Z : Size=0, Index=None, Domain=Any, Default=None, Mutable=False
    Key : Value


The parameters can be indexed:

In [6]:
model.Index = Set(initialize = ['First', 'Second', 'Third'], ordered = True)

During declaration we write the index.

In [7]:
model.ZIndex = Param(model.Index)
model.ZIndex.pprint()

ZIndex : Size=0, Index=Index, Domain=Any, Default=None, Mutable=False
    Key : Value


In order to associate values, we can initialize using a dictionary

In [8]:
model.ZIni = Param(model.Index, initialize = {'First':32, 'Second': 23,'Third': 21})
model.ZIni.pprint()

ZIni : Size=3, Index=Index, Domain=Any, Default=None, Mutable=False
    Key    : Value
     First :    32
    Second :    23
     Third :    21


It can be done with a rule.

In [9]:
def Initialization(model, i):
    return i + ' Parametro'
model.ZRule = Param(model.Index, initialize = Initialization)
model.ZRule.pprint()

ZRule : Size=3, Index=Index, Domain=Any, Default=None, Mutable=False
    Key    : Value
     First : First Parametro
    Second : Second Parametro
     Third : Third Parametro


If the set is ordered, it can be done refering to previous parameters:

In [10]:
def Previo(model, i):
    if i == 'First':
        return 'First' + i
    return i + model.ZPrevio['First']
model.ZPrevio = Param(model.Index, initialize = Previo)
model.ZPrevio.pprint()

ZPrevio : Size=3, Index=Index, Domain=Any, Default=None, Mutable=False
    Key    : Value
     First :       FirstFirst
    Second : SecondFirstFirst
     Third :  ThirdFirstFirst


Parameters can hold any type of data. The default domain is `Any`. We can establish the domain manually in order to check that we are not violating it.

In [11]:
model.ZReal = Param(within = Reals)

## Mutability

While normally parameters represent constant values, they can be declared as mutable. For example:

$$ x_{1} + 17x_{2} - 4x_{3}$$

In [12]:
model = ConcreteModel()
model.A = Set(initialize = [1,2,3])
model.P = Param(model.A, initialize = {1:1, 2:17, 3:-4})
model.x = Var(model.A, within = NonNegativeReals)
def Obj_Rule(model):
    return sum(model.P[i]*model.x[i] for i in model.A)
model.Obj = Objective(rule = Obj_Rule)

In [13]:
model.Obj.pprint()

Obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : x[1] + 17*x[2] - 4*x[3]


We can see that it substitued the value of the parameters. If we now declare them as mutable.

In [14]:
model = ConcreteModel()
model.A = Set(initialize = [1,2,3])
model.P = Param(model.A, initialize = {1:1, 2:17, 3:-4}, mutable = True)
model.x = Var(model.A, within = NonNegativeReals)
def Obj_Rule(model):
    return sum(model.P[i]*model.x[i] for i in model.A)
model.Obj = Objective(rule = Obj_Rule)

In [15]:
model.Obj.pprint()

Obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : P[1]*x[1] + P[2]*x[2] + P[3]*x[3]


The value is not substitued at initialization. It can still be changed by:

In [16]:
model.P[2] = 4.2